### Sequential Chain in LangChain and Databricks Agent

![sequential_chain](./Assets/sequential_chain.png)

### Installing Utilities and Libraries

In [ ]:
%pip install databricks-langchain==0.12.1 langchain-community==0.4.1 langchain-experimental==0.4.1

### Restarting our Python Kernel

In [ ]:
dbutils.library.restartPython()

### Creating the LLM Component

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from databricks_langchain import ChatDatabricks
import os

llm = ChatDatabricks(
    endpoint="databricks-claude-sonnet-4-5",
    temperature=0.1,
)

### Creating the Topics Builder Agent Prompt Template

In [ ]:
topics_builder_agent_prompt_template = PromptTemplate(
    input_variables = ["user_query"],
    template = """You are the Topic Builder Agent, the first step in a sequential workflow that generates a personalized learning roadmap based on a user’s goals.

Your task is to take the user's learning goals and produce a structured, actionable set of foundational outputs that will be used by subsequent workflow steps.
You must analyze the user’s intent, experience level, target role, timeline, and constraints to generate topics, subtopics, relevant technologies, and prerequisite skills required to achieve the learning objective. The topics generated should be aligned with Microsoft Tech Stack and MS Learn modules. You work is to only generate topics and nothing else to pass onto the ms learn module picker agent.

The output format should be as follows:
User Goal: Summary of the user’s learning goal.
Topics:
- Topic 1
  - Subtopic 1.1
  - Subtopic 1.2
  
The user query is: {user_query}"""
)

### Creating the Module Picker Agent Propt Template

In [ ]:
module_picker_agent_prompt_template = PromptTemplate(
    input_variables = ["topics"],
    template = """You are the MS Learn Module Picker Agent, the second step in a sequential workflow that builds a personalized Microsoft Learn study path.

You receive structured topics generated by the Topic Builder Agent.
Your job is to map each topic and subtopic to the most relevant Microsoft Learn modules, learning paths, and units available in the Microsoft Learn catalog.

Return a structured list of topics with their corresponding Microsoft Learn modules, including module names, URLs, and brief descriptions.

the topics are: {topics}"""
)

### Creating the Study Plan Generator Agent Prompt Template

In [ ]:
study_plan_generator_agent_prompt_template = PromptTemplate(
    input_variables=["modules"],
    template="""You are the Study Plan Generator Agent, the third step in a sequential workflow.

You receive curated Microsoft Learn modules (with URLs, descriptions, and subtopic mapping) and the user’s timeline information.
Your job is to convert these modules into a structured, realistic, and achievable week-by-week study plan, factoring in the user’s available time, pace, module complexity, and learning depth.

Your output MUST be clean, human friendly.

the modules are: {modules}"""
)

### Chaining all the Sequential Steps

In [ ]:
from langchain_core.runnables import RunnableSequence

# creating individual chains
topic_generator_chain = topics_builder_agent_prompt_template | llm
module_selection_chain = module_picker_agent_prompt_template | llm
study_plan_generator_chain = study_plan_generator_agent_prompt_template | llm

# synthesizing all the chains in a sequential order
ms_learn_path_builder = RunnableSequence(topic_generator_chain, module_selection_chain, study_plan_generator_chain)

### Invoking the Runnable Chain

In [ ]:
result = ms_learn_path_builder.invoke(
    {
        "user_query": "Help me become an Azure AI Engineer in 3 months"
    }
)

print("Result: \n")
print(result.content)